In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
import logging

# إعداد التسجيل
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')


class AqarmapScraper:
    def __init__(self):
        self.session = requests.Session()
        self.set_headers()
        self.properties_data = []

    def set_headers(self):
        """إعداد headers لمحاكاة المتصفح"""
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'ar,en;q=0.9,en-US;q=0.8',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        }

    def extract_price(self, price_text):
        """استخراج السعر من النص"""
        try:
            # إزالة المسافات والأحرف غير المرغوب فيها
            cleaned_price = re.sub(r'[^\d]', '', price_text.strip())
            return int(cleaned_price) if cleaned_price else None
        except:
            return None

    def extract_area(self, area_text):
        """استخراج المساحة من النص"""
        try:
            area_match = re.search(r'(\d+)\s*متر', area_text)
            return int(area_match.group(1)) if area_match else None
        except:
            return None

    def extract_price_per_meter(self, price_meter_text):
        """استخراج سعر المتر من النص"""
        try:
            price_match = re.search(r'(\d+[,\d]*)\s*ج/ م²', price_meter_text)
            if price_match:
                return int(re.sub(r'[^\d]', '', price_match.group(1)))
            return None
        except:
            return None

    def extract_property_data(self, listing_card):
        """استخراج بيانات العقار من البطاقة"""
        try:
            property_data = {}

            # استخراج العنوان والرابط
            title_elem = listing_card.find('h2', class_='text-body_2')
            property_data['title'] = title_elem.get(
                'title', '').strip() if title_elem else ''

            # استخراج الرابط
            link_elem = listing_card.find('a', href=True)
            property_data['link'] = 'https://aqarmap.com.eg' + \
                link_elem['href'] if link_elem else ''

            # استخراج السعر
            price_elem = listing_card.find('span', class_='text-title_4')
            if price_elem:
                price_text = ' '.join(price_elem.stripped_strings)
                property_data['price'] = self.extract_price(price_text)

            # استخراج سعر المتر
            price_per_meter_elem = listing_card.find(
                'span', class_='text-caption', string=re.compile(r'ج/ م²'))
            if price_per_meter_elem:
                property_data['price_per_meter'] = self.extract_price_per_meter(
                    price_per_meter_elem.text)

            # استخراج الموقع
            location_elem = listing_card.find(
                'p', class_='text-caption', attrs={'title': True})
            property_data['location'] = location_elem.get(
                'title', '') if location_elem else ''

            # استخراج المساحة والغرف والحمامات
            details_container = listing_card.find(
                'div', class_='flex items-center gap-x-2x')
            if details_container:
                details = details_container.find_all(
                    'p', class_='text-caption')
                for detail in details:
                    text = detail.text.strip()
                    if 'متر²' in text:
                        property_data['area'] = self.extract_area(text)
                    elif 'غرف' in text or re.search(r'\d+\s*$', text):
                        property_data['bedrooms'] = re.search(r'(\d+)', text)
                        property_data['bedrooms'] = int(property_data['bedrooms'].group(
                            1)) if property_data['bedrooms'] else None
                    elif 'حمام' in text:
                        property_data['bathrooms'] = re.search(r'(\d+)', text)
                        property_data['bathrooms'] = int(property_data['bathrooms'].group(
                            1)) if property_data['bathrooms'] else None

            # استخراج عدد الصور
            images_elem = listing_card.find(
                'span', class_='text-white', string=re.compile(r'\d+'))
            property_data['images_count'] = int(
                images_elem.text) if images_elem else 0

            # استخراج وسائل الاتصال
            whatsapp_btn = listing_card.find(
                'button', string=re.compile(r'واتساب'))
            call_btn = listing_card.find('button', string=re.compile(r'اتصال'))
            property_data['has_whatsapp'] = whatsapp_btn is not None
            property_data['has_call'] = call_btn is not None

            # إضافة timestamp
            property_data['scraped_at'] = datetime.now().strftime(
                '%Y-%m-%d %H:%M:%S')

            return property_data

        except Exception as e:
            logging.error(f"Error extracting property data: {e}")
            return None

    def scrape_page(self, url):
        """استخراج البيانات من صفحة واحدة"""
        try:
            logging.info(f"Scraping page: {url}")

            response = self.session.get(url, headers=self.headers, timeout=30)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # العثور على جميع بطاقات العقارات
            listing_cards = soup.find_all('div', class_='listing-card')
            logging.info(f"Found {len(listing_cards)} property cards")

            for card in listing_cards:
                property_data = self.extract_property_data(card)
                if property_data:
                    self.properties_data.append(property_data)
                    logging.info(
                        f"Extracted: {property_data.get('title', 'N/A')}")

            # التحقق من وجود صفحة تالية
            next_page = soup.find('a', href=True, string='2')
            return next_page['href'] if next_page else None

        except requests.RequestException as e:
            logging.error(f"Request error: {e}")
            return None
        except Exception as e:
            logging.error(f"Scraping error: {e}")
            return None

    def scrape_all_pages(self, base_url, max_pages=18):
        """استخراج البيانات من جميع الصفحات"""
        current_url = base_url
        page_count = 1

        while current_url and page_count <= max_pages:
            logging.info(f"Scraping page {page_count}")

            next_page_url = self.scrape_page(current_url)

            if next_page_url:
                current_url = 'https://aqarmap.com.eg' + next_page_url
            else:
                break

            page_count += 1
            time.sleep(2)  # احترام الموقع

        logging.info(
            f"Scraping completed. Total properties: {len(self.properties_data)}")

    def save_to_excel(self, filename='aqarmap_properties.xlsx'):
        """حفظ البيانات في ملف Excel"""
        if not self.properties_data:
            logging.warning("No data to save")
            return

        df = pd.DataFrame(self.properties_data)

        # تنظيف الأعمدة
        columns_order = [
            'title', 'price', 'price_per_meter', 'area', 'bedrooms',
            'bathrooms', 'location', 'images_count', 'has_whatsapp',
            'has_call', 'link', 'scraped_at'
        ]

        # إعادة ترتيب الأعمدة المتوفرة فقط
        available_columns = [col for col in columns_order if col in df.columns]
        df = df[available_columns]

        # حفظ في Excel
        df.to_excel(filename, index=False, engine='openpyxl')
        logging.info(f"Data saved to {filename}")

        # عرض إحصائيات
        self.show_statistics(df)

    def show_statistics(self, df):
        """عرض إحصائيات عن البيانات"""
        print("\n" + "="*50)
        print("إحصائيات البيانات المستخرجة:")
        print("="*50)
        print(f"إجمالي العقارات: {len(df)}")
        print(
            f"متوسط السعر: {df['price'].mean():,.0f} جنيه" if 'price' in df.columns else "لا توجد بيانات سعر")
        print(
            f"متوسط المساحة: {df['area'].mean():,.0f} متر²" if 'area' in df.columns else "لا توجد بيانات مساحة")
        print(
            f"العقارات المتاحة عبر واتساب: {df['has_whatsapp'].sum()}" if 'has_whatsapp' in df.columns else "")
        print(
            f"العقارات المتاحة عبر اتصال: {df['has_call'].sum()}" if 'has_call' in df.columns else "")


def main():
    """الدالة الرئيسية"""
    scraper = AqarmapScraper()

    # URL المطلوب
    url = "https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1"

    try:
        # استخراج البيانات من 5 صفحات كحد أقصى
        scraper.scrape_all_pages(url, max_pages=5)

        # حفظ في Excel
        scraper.save_to_excel('عقارات_القاهرة_الجديدة.xlsx')

        print("\n✅ تم الانتهاء من استخراج البيانات بنجاح!")

    except Exception as e:
        logging.error(f"Main execution error: {e}")


if __name__ == "__main__":
    main()

2025-09-29 23:29:51,579 - INFO - Scraping page 1
2025-09-29 23:29:51,581 - INFO - Scraping page: https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1
2025-09-29 23:29:53,409 - INFO - Found 22 property cards
2025-09-29 23:29:53,410 - INFO - Extracted: فيلا مستقلة ٢٦٠ متر ، كمبوند ذا وندر مارك
2025-09-29 23:29:53,411 - INFO - Extracted: 
2025-09-29 23:29:53,413 - INFO - Extracted: شقة راقية ببيت الوطن  الحي الثامن ثاني صف من محور بن زايد
2025-09-29 23:29:53,415 - INFO - Extracted: عيادة للبيع بجوار الجامعة الأمريكية
2025-09-29 23:29:53,416 - INFO - Extracted: شقه بحديقه خلفي +مخزن
2025-09-29 23:29:53,417 - INFO - Extracted: ? *فرصة لا تُعوّض لامتلاك شقة مميزة في مدينتي – القاهرة الجديدة!*
2025-09-29 23:29:53,418 - INFO - Extracted: فرصه ممتازه - شقة 208 م الحي الرابع خطوات من النادي الاهلي - بيت الوطن
2025-09-29 23:29:53,419 - INFO - Extracted: شقة للبيع نايل بوليفارد
2025-09-29 23:29:53,421 - INFO - Extracted: شقة للبيع في مدينتي
2025-09-29 23:29:53,422 - INFO


إحصائيات البيانات المستخرجة:
إجمالي العقارات: 106
متوسط السعر: 15,551,063 جنيه
متوسط المساحة: 160 متر²
العقارات المتاحة عبر واتساب: 0
العقارات المتاحة عبر اتصال: 0

✅ تم الانتهاء من استخراج البيانات بنجاح!


In [12]:
import time
import re
import logging
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# ------------------------------
# Logging
# ------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)


class AqarMapScraper:
    def __init__(self, base_url, max_pages=5, wait_time=3):
        self.base_url = base_url
        self.max_pages = max_pages
        self.wait_time = wait_time
        self.results = []
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()))
        self.run_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    def clean_number(self, text):
        """تنضيف أي نص عشان يطلع منه أرقام"""
        try:
            return float(re.sub(r"[^\d.]", "", text)) if text else None
        except:
            return None

    def scrape_page(self, url):
        self.driver.get(url)
        time.sleep(self.wait_time)  # نستنى الـ JS يحمّل
        soup = BeautifulSoup(self.driver.page_source, "html.parser")

        properties = soup.find_all("div", class_=re.compile("card__container"))
        logging.info(f"Scraped {len(properties)} properties from {url}")

        for prop in properties:
            title = prop.find("h2")
            price = prop.find(string=re.compile("EGP"))
            area = prop.find(string=re.compile("متر"))
            location = prop.find("div", class_=re.compile("card__location"))

            data = {
                "Title": title.get_text(strip=True) if title else None,
                "Price": self.clean_number(price),
                "Area (sqm)": self.clean_number(area),
                "Location": location.get_text(strip=True) if location else None,
                "URL": url,
                "Scraped At": self.run_timestamp
            }
            self.results.append(data)

        # نجيب لينك الصفحة اللي بعدها
        next_page = soup.find("a", class_="page-link",
                              string=re.compile(r"^\d+$"))
        if next_page:
            return "https://aqarmap.com.eg" + next_page["href"]
        return None

    def scrape(self):
        url = self.base_url
        for page in range(1, self.max_pages + 1):
            if not url:
                break
            logging.info(f"Scraping page {page}: {url}")
            url = self.scrape_page(url)

        self.driver.quit()
        logging.info("Finished scraping.")

    def save_to_excel(self, filename="aqarmap_properties.xlsx"):
        df = pd.DataFrame(self.results)
        with pd.ExcelWriter(filename, engine="openpyxl") as writer:
            df.to_excel(writer, sheet_name="Properties", index=False)
        logging.info(f"Data saved to {filename}")


# ------------------------------
# Run Script
# ------------------------------
if __name__ == "__main__":
    base_url = "https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1"
    scraper = AqarMapScraper(base_url, max_pages=3,
                             wait_time=4)  # ممكن تزود max_pages
    scraper.scrape()
    scraper.save_to_excel()

2025-09-29 23:34:04,230 - INFO - ====== WebDriver manager ======
2025-09-29 23:34:05,062 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-29 23:34:05,267 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-29 23:34:05,455 - INFO - Driver [C:\Users\ASUS\.wdm\drivers\chromedriver\win64\140.0.7339.207\chromedriver-win32/chromedriver.exe] found in cache
2025-09-29 23:34:07,381 - INFO - Scraping page 1: https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1
2025-09-29 23:34:14,534 - INFO - Scraped 0 properties from https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1
2025-09-29 23:34:16,686 - INFO - Finished scraping.
2025-09-29 23:34:16,722 - INFO - Data saved to aqarmap_properties.xlsx


In [14]:
import time
import re
import logging
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# ------------------------------
# Logging
# ------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)


class AqarMapScraper:
    def __init__(self, base_url, max_pages=5, wait_time=3):
        self.base_url = base_url
        self.max_pages = max_pages
        self.wait_time = wait_time
        self.results = []
        self.run_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.driver = self.setup_driver()

    def setup_driver(self):
        """إعداد متصفح Chrome مع خيارات محسنة"""
        chrome_options = Options()
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument(
            "--disable-blink-features=AutomationControlled")
        chrome_options.add_experimental_option(
            "excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        chrome_options.add_argument("--lang=ar")

        driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=chrome_options
        )
        driver.execute_script(
            "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        return driver

    def clean_number(self, text):
        """تنظيف النص لاستخراج الأرقام فقط"""
        if not text:
            return None
        try:
            # إزالة كل ما ليس رقماً أو نقطة
            cleaned = re.sub(r"[^\d.]", "", str(text))
            return float(cleaned) if cleaned else None
        except:
            return None

    def extract_price(self, prop):
        """استخراج السعر من العقار"""
        try:
            # البحث عن السعر في عدة أماكن محتملة
            price_selectors = [
                ".text-title_4",
                "[class*='price']",
                "span[class*='جني']",
                "span:contains('جنيه')"
            ]

            for selector in price_selectors:
                price_elem = prop.select_one(selector)
                if price_elem:
                    price_text = price_elem.get_text(strip=True)
                    # البحث عن الأرقام في النص
                    numbers = re.findall(r'[\d,]+', price_text)
                    if numbers:
                        return self.clean_number(numbers[0])

            return None
        except Exception as e:
            logging.error(f"Error extracting price: {e}")
            return None

    def extract_area(self, prop):
        """استخراج المساحة"""
        try:
            area_selectors = [
                "[class*='size']",
                "[class*='مساحة']",
                "p:contains('متر')",
                "div:contains('متر²')"
            ]

            for selector in area_selectors:
                area_elem = prop.select_one(selector)
                if area_elem:
                    area_text = area_elem.get_text()
                    # البحث عن الرقم قبل كلمة متر
                    area_match = re.search(r'(\d+)\s*متر', area_text)
                    if area_match:
                        return int(area_match.group(1))

            return None
        except Exception as e:
            logging.error(f"Error extracting area: {e}")
            return None

    def extract_location(self, prop):
        """استخراج الموقع"""
        try:
            location_selectors = [
                "[class*='location']",
                "[class*='موقع']",
                ".text-caption[title]",
                "p:contains('القاهرة')"
            ]

            for selector in location_selectors:
                loc_elem = prop.select_one(selector)
                if loc_elem:
                    # الحصول على النص أو title attribute
                    location = loc_elem.get(
                        'title') or loc_elem.get_text(strip=True)
                    if location and len(location) > 3:
                        return location

            return "غير محدد"
        except Exception as e:
            logging.error(f"Error extracting location: {e}")
            return "غير محدد"

    def extract_title(self, prop):
        """استخراج عنوان العقار"""
        try:
            title_selectors = [
                "h2",
                "h3",
                "[class*='title']",
                "[class*='عنوان']",
                ".text-body_2"
            ]

            for selector in title_selectors:
                title_elem = prop.select_one(selector)
                if title_elem:
                    title = title_elem.get_text(strip=True)
                    if title and len(title) > 5:
                        return title

            return "لا يوجد عنوان"
        except Exception as e:
            logging.error(f"Error extracting title: {e}")
            return "لا يوجد عنوان"

    def extract_details(self, prop):
        """استخراج تفاصيل إضافية مثل الغرف والحمامات"""
        try:
            details = {"غرف": None, "حمامات": None, "صور": None}

            # البحث عن عدد الغرف
            bedroom_elem = prop.find(string=re.compile(
                r'غرف|غرفة|bedroom', re.IGNORECASE))
            if bedroom_elem:
                bed_match = re.search(
                    r'(\d+)\s*غرف?', bedroom_elem, re.IGNORECASE)
                if bed_match:
                    details["غرف"] = int(bed_match.group(1))

            # البحث عن عدد الحمامات
            bath_elem = prop.find(string=re.compile(
                r'حمام|bathroom', re.IGNORECASE))
            if bath_elem:
                bath_match = re.search(
                    r'(\d+)\s*حمام?', bath_elem, re.IGNORECASE)
                if bath_match:
                    details["حمامات"] = int(bath_match.group(1))

            # البحث عن عدد الصور
            images_elem = prop.find(
                string=re.compile(r'صور|image', re.IGNORECASE))
            if images_elem:
                img_match = re.search(r'(\d+)', images_elem)
                if img_match:
                    details["صور"] = int(img_match.group(1))

            return details
        except Exception as e:
            logging.error(f"Error extracting details: {e}")
            return {"غرف": None, "حمامات": None, "صور": None}

    def scrape_page(self, url):
        """استخراج البيانات من صفحة واحدة"""
        try:
            self.driver.get(url)
            WebDriverWait(self.driver, self.wait_time).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            time.sleep(2)  # انتظار إضافي للتأكد من تحميل المحتوى

            soup = BeautifulSoup(self.driver.page_source, "html.parser")

            # البحث عن بطاقات العقارات باستخدام أنماط مختلفة
            property_selectors = [
                "div.listing-card",
                "div[class*='card']",
                "div.property-item",
                "div.listing-item"
            ]

            properties = []
            for selector in property_selectors:
                properties = soup.select(selector)
                if properties:
                    logging.info(
                        f"Found {len(properties)} properties using selector: {selector}")
                    break

            if not properties:
                logging.warning("No properties found on page")
                return None

            for prop in properties:
                try:
                    title = self.extract_title(prop)
                    price = self.extract_price(prop)
                    area = self.extract_area(prop)
                    location = self.extract_location(prop)
                    details = self.extract_details(prop)

                    # استخراج الرابط إذا وجد
                    link_elem = prop.find('a', href=True)
                    property_url = "https://aqarmap.com.eg" + \
                        link_elem['href'] if link_elem else url

                    data = {
                        "العنوان": title,
                        "السعر (جنيه)": price,
                        "المساحة (م²)": area,
                        "الموقع": location,
                        "عدد الغرف": details["غرف"],
                        "عدد الحمامات": details["حمامات"],
                        "عدد الصور": details["صور"],
                        "رابط العقار": property_url,
                        "وقت الاستخراج": self.run_timestamp
                    }

                    self.results.append(data)
                    logging.info(f"تم استخراج: {title}")

                except Exception as e:
                    logging.error(f"Error processing property: {e}")
                    continue

            # البحث عن رابط الصفحة التالية
            next_page = soup.find("a", href=True, string=re.compile(
                r"التالي|Next|›|>", re.IGNORECASE))
            if not next_page:
                # البحث عن أرقام الصفحات
                next_page = soup.find(
                    "a", href=True, string=re.compile(r"^\d+$"))

            if next_page:
                next_url = next_page['href']
                if not next_url.startswith('http'):
                    next_url = "https://aqarmap.com.eg" + next_url
                return next_url

            return None

        except Exception as e:
            logging.error(f"Error scraping page {url}: {e}")
            return None

    def scrape(self):
        """بدء عملية الاستخراج"""
        url = self.base_url
        for page in range(1, self.max_pages + 1):
            if not url:
                break
            logging.info(f"جاري استخراج الصفحة {page}: {url}")
            next_url = self.scrape_page(url)

            if next_url and next_url != url:
                url = next_url
            else:
                break

            # انتظار بين الصفحات
            time.sleep(3)

        self.driver.quit()
        logging.info(
            f"تم الانتهاء من الاستخراج. العدد الإجمالي: {len(self.results)} عقار")

    def save_to_excel(self, filename="عقارات_القاهرة_الجديدة.xlsx"):
        """حفظ البيانات في ملف Excel بعناوين عربية"""
        if not self.results:
            logging.warning("لا توجد بيانات لحفظها")
            return

        df = pd.DataFrame(self.results)

        # تنظيف البيانات
        df = df.drop_duplicates(subset=['العنوان', 'السعر (جنيه)', 'الموقع'])

        # إضافة عمود للملاحظات
        df['ملاحظات'] = ''

        # إعادة ترتيب الأعمدة
        column_order = [
            'العنوان',
            'السعر (جنيه)',
            'المساحة (م²)',
            'الموقع',
            'عدد الغرف',
            'عدد الحمامات',
            'عدد الصور',
            'رابط العقار',
            'ملاحظات',
            'وقت الاستخراج'
        ]

        # التأكد من وجود جميع الأعمدة
        for col in column_order:
            if col not in df.columns:
                df[col] = None

        df = df[column_order]

        try:
            with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='العقارات', index=False)

                # تحسين عرض الأعمدة
                worksheet = writer.sheets['العقارات']
                for idx, col in enumerate(df.columns):
                    max_length = max(df[col].astype(
                        str).str.len().max(), len(col)) + 2
                    worksheet.column_dimensions[chr(
                        65 + idx)].width = min(max_length, 50)

            logging.info(f"تم حفظ البيانات في ملف: {filename}")

            # عرض إحصائيات
            self.show_statistics(df)

        except Exception as e:
            logging.error(f"Error saving to Excel: {e}")

    def show_statistics(self, df):
        """عرض إحصائيات عن البيانات المستخرجة"""
        print("\n" + "="*60)
        print("📊 إحصائيات البيانات المستخرجة")
        print("="*60)
        print(f"🏠 إجمالي العقارات: {len(df)}")

        if 'السعر (جنيه)' in df.columns:
            avg_price = df['السعر (جنيه)'].mean()
            if pd.notna(avg_price):
                print(f"💰 متوسط السعر: {avg_price:,.0f} جنيه")

        if 'المساحة (م²)' in df.columns:
            avg_area = df['المساحة (م²)'].mean()
            if pd.notna(avg_area):
                print(f"📐 متوسط المساحة: {avg_area:.0f} متر مربع")

        if 'عدد الغرف' in df.columns:
            rooms_count = df['عدد الغرف'].count()
            print(
                f"🛏️ عدد العقارات التي تحتوي على بيانات الغرف: {rooms_count}")

        print("="*60)


# ------------------------------
# تشغيل السكريبت
# ------------------------------
if __name__ == "__main__":
    base_url = "https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1"

    try:
        scraper = AqarMapScraper(base_url, max_pages=3, wait_time=4)
        scraper.scrape()
        scraper.save_to_excel()

        print("\n🎉 تم الانتهاء من عملية الاستخراج بنجاح!")
        print("📁 تم حفظ البيانات في ملف 'عقارات_القاهرة_الجديدة.xlsx'")

    except Exception as e:
        logging.error(f"خطأ في التشغيل الرئيسي: {e}")
    finally:
        if 'scraper' in locals():
            scraper.driver.quit()

2025-09-29 23:37:56,907 - INFO - ====== WebDriver manager ======
2025-09-29 23:37:57,829 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-29 23:37:58,016 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-29 23:37:58,207 - INFO - Driver [C:\Users\ASUS\.wdm\drivers\chromedriver\win64\140.0.7339.207\chromedriver-win32/chromedriver.exe] found in cache
2025-09-29 23:37:59,337 - INFO - جاري استخراج الصفحة 1: https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1
2025-09-29 23:38:04,969 - INFO - Found 22 properties using selector: div.listing-card
c:\Users\ASUS\anaconda3\Lib\site-packages\soupsieve\css_parser.py:856: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028
2025-09-29 23:38:04,976 - INFO - تم استخراج: فيلا مستقلة ٢٦٠ متر ، كمبوند ذا وندر مارك
2025-09-29 23:38:04,978 - INFO - تم استخراج: 65,000,000
    جنيه
2025-09-29 23:38:04,980 - INFO 


📊 إحصائيات البيانات المستخرجة
🏠 إجمالي العقارات: 22
💰 متوسط السعر: 11,937,033 جنيه
📐 متوسط المساحة: 161 متر مربع
🛏️ عدد العقارات التي تحتوي على بيانات الغرف: 0

🎉 تم الانتهاء من عملية الاستخراج بنجاح!
📁 تم حفظ البيانات في ملف 'عقارات_القاهرة_الجديدة.xlsx'


2025-09-29 23:38:11,322 - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000271CC5E7F00>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it')': /session/5e46102e64824734c3ae107193d54a98
2025-09-29 23:38:15,414 - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000271CC5E7CE0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it')': /session/5e46102e64824734c3ae107193d54a98
2025-09-29 23:38:19,503 - WARNING - Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPCon

In [17]:
import time
import re
import logging
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os

# ------------------------------
# Logging
# ------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)


class AqarMapScraper:
    def __init__(self, base_url, max_pages=3, wait_time=5):
        self.base_url = base_url
        self.max_pages = max_pages
        self.wait_time = wait_time
        self.results = []
        self.run_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.driver = self.setup_driver()

    def setup_driver(self):
        """إعداد متصفح Chrome بطريقة أكثر موثوقية"""
        try:
            chrome_options = Options()

            # إعدادات لتحسين الأداء والموثوقية
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument(
                "--disable-blink-features=AutomationControlled")
            chrome_options.add_experimental_option(
                "excludeSwitches", ["enable-automation"])
            chrome_options.add_experimental_option(
                'useAutomationExtension', False)
            chrome_options.add_argument("--lang=ar-EG")
            chrome_options.add_argument(
                "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

            # إعدادات لتحسين الاستقرار
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--disable-extensions")
            chrome_options.add_argument("--disable-notifications")
            chrome_options.add_argument("--disable-popup-blocking")

            # استخدام Chrome الموجود على النظام (بدون webdriver-manager)
            try:
                driver = webdriver.Chrome(options=chrome_options)
            except Exception as e:
                logging.warning(
                    f"ChDriver not found in PATH, trying direct path...")
                # محاولة المسار المباشر
                chrome_paths = [
                    "C:/Program Files/Google/Chrome/Application/chrome.exe",
                    "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"
                ]
                for path in chrome_paths:
                    if os.path.exists(path):
                        chrome_options.binary_location = path
                        break

                driver = webdriver.Chrome(options=chrome_options)

            # إعدادات إضافية
            driver.set_page_load_timeout(30)
            driver.implicitly_wait(10)

            logging.info("تم إعداد المتصفح بنجاح")
            return driver

        except Exception as e:
            logging.error(f"فشل في إعداد المتصفح: {e}")
            raise

    def wait_for_page_load(self, timeout=30):
        """انتظار تحميل الصفحة"""
        try:
            WebDriverWait(self.driver, timeout).until(
                lambda driver: driver.execute_script(
                    "return document.readyState") == "complete"
            )
            return True
        except:
            logging.warning("انتهت مهلة انتظار تحميل الصفحة")
            return False

    def extract_property_data(self, prop):
        """استخراج بيانات العقار من البطاقة"""
        try:
            data = {
                "العنوان": "غير معروف",
                "السعر (جنيه)": None,
                "المساحة (م²)": None,
                "الموقع": "غير معروف",
                "عدد الغرف": None,
                "عدد الحمامات": None,
                "عدد الصور": None,
                "رابط العقار": self.base_url,
                "وقت الاستخراج": self.run_timestamp
            }

            # استخراج العنوان
            try:
                title_elem = prop.find('h2') or prop.find('h3') or prop.find(
                    attrs={'class': re.compile(r'title|heading')})
                if title_elem:
                    data["العنوان"] = title_elem.get_text(strip=True)
            except:
                pass

            # استخراج السعر
            try:
                price_text = prop.get_text()
                price_match = re.search(r'(\d[\d,\.]+)\s*جني?ه?', price_text)
                if price_match:
                    price_clean = re.sub(r'[^\d]', '', price_match.group(1))
                    data["السعر (جنيه)"] = int(
                        price_clean) if price_clean else None
            except:
                pass

            # استخراج المساحة
            try:
                area_match = re.search(r'(\d+)\s*متر', prop.get_text())
                if area_match:
                    data["المساحة (م²)"] = int(area_match.group(1))
            except:
                pass

            # استخراج الموقع
            try:
                location_elems = prop.find_all(
                    text=re.compile(r'القاهرة|التجمع|الحي|شارع'))
                if location_elems:
                    data["الموقع"] = location_elems[0].strip()
            except:
                pass

            # استخراج عدد الغرف والحمامات
            try:
                # البحث عن نمط مثل "3 غرف" أو "2 حمام"
                rooms_match = re.search(r'(\d+)\s*غرف?', prop.get_text())
                if rooms_match:
                    data["عدد الغرف"] = int(rooms_match.group(1))

                baths_match = re.search(r'(\d+)\s*حمام?', prop.get_text())
                if baths_match:
                    data["عدد الحمامات"] = int(baths_match.group(1))
            except:
                pass

            # استخراج عدد الصور
            try:
                images_match = re.search(r'(\d+)\s*صور?', prop.get_text())
                if images_match:
                    data["عدد الصور"] = int(images_match.group(1))
            except:
                pass

            # استخراج الرابط
            try:
                link_elem = prop.find('a', href=True)
                if link_elem and link_elem['href']:
                    data["رابط العقار"] = "https://aqarmap.com.eg" + \
                        link_elem['href']
            except:
                pass

            return data

        except Exception as e:
            logging.error(f"خطأ في استخراج بيانات العقار: {e}")
            return None

    def scrape_page(self, url):
        """استخراج البيانات من صفحة واحدة"""
        try:
            logging.info(f"جاري تحميل الصفحة: {url}")
            self.driver.get(url)

            # انتظار تحميل الصفحة
            if not self.wait_for_page_load():
                logging.warning(
                    "لم يتم تحميل الصفحة بالكامل، لكن سنحاول المتابعة")

            time.sleep(3)  # انتظار إضافي

            # الحصول على محتوى الصفحة
            page_source = self.driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")

            # البحث عن بطاقات العقارات بأنماط مختلفة
            property_cards = []

            # محاولة أنماط CSS مختلفة
            selectors = [
                "div.listing-card",
                "div[class*='card']",
                "div[class*='property']",
                "div[class*='listing']",
                "article",
                ".search-result-item"
            ]

            for selector in selectors:
                cards = soup.select(selector)
                if cards:
                    logging.info(
                        f"تم العثور على {len(cards)} عقار باستخدام: {selector}")
                    property_cards.extend(cards)
                    break

            if not property_cards:
                # إذا لم نجد بأي selector، نبحث عن أي div يحتوي على بيانات عقارية
                all_divs = soup.find_all('div')
                for div in all_divs:
                    text = div.get_text()
                    if any(keyword in text for keyword in ['جنيه', 'متر', 'غرف', 'حمام']):
                        property_cards.append(div)

                if property_cards:
                    logging.info(
                        f"تم العثور على {len(property_cards)} عقار بالبحث النصي")

            if not property_cards:
                logging.warning("لم يتم العثور على أي عقارات في هذه الصفحة")
                # حفظ صفحة HTML للتdebug
                with open("debug_page.html", "w", encoding="utf-8") as f:
                    f.write(page_source)
                return None

            # استخراج البيانات من كل بطاقة
            successful_extractions = 0
            for card in property_cards:
                property_data = self.extract_property_data(card)
                if property_data and property_data["العنوان"] != "غير معروف":
                    self.results.append(property_data)
                    successful_extractions += 1
                    logging.info(f"تم استخراج: {property_data['العنوان']}")

            logging.info(
                f"تم استخراج {successful_extractions} عقار بنجاح من هذه الصفحة")

            # البحث عن الصفحة التالية
            next_url = self.find_next_page(soup)
            return next_url

        except Exception as e:
            logging.error(f"خطأ في استخراج الصفحة: {e}")
            return None

    def find_next_page(self, soup):
        """البحث عن رابط الصفحة التالية"""
        try:
            # البحث بأنماط مختلفة
            next_selectors = [
                "a[class*='next']",
                "a[class*='page']",
                "a:contains('التالي')",
                "a:contains('Next')",
                "a[aria-label*='next']",
                "a[title*='next']"
            ]

            for selector in next_selectors:
                next_link = soup.select_one(selector)
                if next_link and next_link.get('href'):
                    next_url = next_link['href']
                    if not next_url.startswith('http'):
                        next_url = "https://aqarmap.com.eg" + next_url
                    logging.info(f"تم العثور على الصفحة التالية: {next_url}")
                    return next_url

            # البحث في أرقام الصفحات
            page_links = soup.find_all(
                'a', href=True, string=re.compile(r'^\d+$'))
            current_page_match = re.search(
                r'page=(\d+)', self.driver.current_url)
            if current_page_match and page_links:
                current_page = int(current_page_match.group(1))
                for link in page_links:
                    if link.text.isdigit():
                        page_num = int(link.text)
                        if page_num == current_page + 1:
                            next_url = link['href']
                            if not next_url.startswith('http'):
                                next_url = "https://aqarmap.com.eg" + next_url
                            return next_url

            logging.info("لم يتم العثور على صفحة تالية")
            return None

        except Exception as e:
            logging.error(f"خطأ في البحث عن الصفحة التالية: {e}")
            return None

    def scrape(self):
        """بدء عملية الاستخراج"""
        try:
            url = self.base_url
            for page_num in range(1, self.max_pages + 1):
                if not url:
                    break

                logging.info(f"🎯 الصفحة {page_num}/{self.max_pages}")
                next_url = self.scrape_page(url)

                if next_url and next_url != url:
                    url = next_url
                else:
                    break

                # انتظار بين الصفحات
                if page_num < self.max_pages:
                    time.sleep(4)

            logging.info(
                f"✅ تم الانتهاء من الاستخراج. العدد الإجمالي: {len(self.results)} عقار")

        except Exception as e:
            logging.error(f"خطأ في عملية الاستخراج: {e}")
        finally:
            if hasattr(self, 'driver'):
                self.driver.quit()
                logging.info("تم إغلاق المتصفح")

    def save_to_excel(self, filename="عقارات_القاهرة_الجديدة.xlsx"):
        """حفظ البيانات في ملف Excel"""
        if not self.results:
            logging.warning("⚠️ لا توجد بيانات لحفظها")
            return

        try:
            df = pd.DataFrame(self.results)

            # تنظيف البيانات
            df = df.drop_duplicates(
                subset=['العنوان', 'السعر (جنيه)'], keep='first')

            # إضافة عمود للملاحظات
            df['ملاحظات'] = ''

            # إعادة ترتيب الأعمدة
            column_order = [
                'العنوان', 'السعر (جنيه)', 'المساحة (م²)', 'الموقع',
                'عدد الغرف', 'عدد الحمامات', 'عدد الصور',
                'رابط العقار', 'ملاحظات', 'وقت الاستخراج'
            ]

            # التأكد من وجود جميع الأعمدة
            for col in column_order:
                if col not in df.columns:
                    df[col] = None

            df = df[column_order]

            # حفظ في Excel
            df.to_excel(filename, index=False, engine='openpyxl')
            logging.info(f"💾 تم حفظ البيانات في: {filename}")

            # عرض إحصائيات
            self.show_statistics(df)

        except Exception as e:
            logging.error(f"خطأ في حفظ الملف: {e}")

    def show_statistics(self, df):
        """عرض إحصائيات عن البيانات"""
        print("\n" + "="*50)
        print("📊 إحصائيات البيانات المستخرجة")
        print("="*50)
        print(f"🏠 إجمالي العقارات: {len(df)}")

        if 'السعر (جنيه)' in df.columns:
            valid_prices = df['السعر (جنيه)'].dropna()
            if len(valid_prices) > 0:
                avg_price = valid_prices.mean()
                print(f"💰 متوسط السعر: {avg_price:,.0f} جنيه")

        if 'المساحة (م²)' in df.columns:
            valid_areas = df['المساحة (م²)'].dropna()
            if len(valid_areas) > 0:
                avg_area = valid_areas.mean()
                print(f"📐 متوسط المساحة: {avg_area:.0f} م²")

        print("="*50)


# ------------------------------
# التشغيل الرئيسي
# ------------------------------
if __name__ == "__main__":
    base_url = "https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1"

    try:
        scraper = AqarMapScraper(base_url, max_pages=1, wait_time=5)
        scraper.scrape()
        scraper.save_to_excel()

        print("\n🎉 تم الانتهاء بنجاح!")

    except Exception as e:
        logging.error(f"❌ فشل التشغيل: {e}")

2025-09-29 23:46:37,394 - INFO - تم إعداد المتصفح بنجاح
2025-09-29 23:46:37,398 - INFO - 🎯 الصفحة 1/1
2025-09-29 23:46:37,399 - INFO - جاري تحميل الصفحة: https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1
2025-09-29 23:46:43,139 - INFO - تم العثور على 22 عقار باستخدام: div.listing-card
C:\Users\ASUS\AppData\Local\Temp\ipykernel_41448\4024536663.py:143: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  location_elems = prop.find_all(
2025-09-29 23:46:43,140 - INFO - تم استخراج: فيلا مستقلة ٢٦٠ متر ، كمبوند ذا وندر مارك
2025-09-29 23:46:43,141 - INFO - تم استخراج: 
2025-09-29 23:46:43,143 - INFO - تم استخراج: شقة راقية ببيت الوطن  الحي الثامن ثاني صف من محور بن زايد
2025-09-29 23:46:43,145 - INFO - تم استخراج: عيادة للبيع بجوار الجامعة الأمريكية
2025-09-29 23:46:43,146 - INFO - تم استخراج: شقه بحديقه خلفي +مخزن
2025-09-29 23:46:43,146 - INFO - تم استخراج: ? *فرصة لا تُعوّض لامتلاك شقة مميزة في مدينتي – القاه


📊 إحصائيات البيانات المستخرجة
🏠 إجمالي العقارات: 21
💰 متوسط السعر: 237,437,033 جنيه
📐 متوسط المساحة: 621 م²

🎉 تم الانتهاء بنجاح!


In [18]:
import time
import re
import logging
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os

# ------------------------------
# Logging
# ------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)


class AqarMapScraper:
    def __init__(self, base_url, max_pages=5, wait_time=5):
        self.base_url = base_url
        self.max_pages = max_pages
        self.wait_time = wait_time
        self.results = []
        self.run_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.driver = self.setup_driver()

    def setup_driver(self):
        """إعداد متصفح Chrome"""
        try:
            chrome_options = Options()
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument(
                "--disable-blink-features=AutomationControlled")
            chrome_options.add_experimental_option(
                "excludeSwitches", ["enable-automation"])
            chrome_options.add_experimental_option(
                'useAutomationExtension', False)
            chrome_options.add_argument("--lang=ar-EG")
            chrome_options.add_argument(
                "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

            driver = webdriver.Chrome(options=chrome_options)
            driver.set_page_load_timeout(30)
            driver.implicitly_wait(10)

            logging.info("تم إعداد المتصفح بنجاح")
            return driver

        except Exception as e:
            logging.error(f"فشل في إعداد المتصفح: {e}")
            raise

    def clean_price(self, price_text):
        """تنظيف نص السعر"""
        if not price_text:
            return None
        try:
            # إزالة كل ما ليس رقماً
            cleaned = re.sub(r'[^\d]', '', str(price_text))
            return int(cleaned) if cleaned else None
        except:
            return None

    def extract_property_data(self, prop):
        """استخراج بيانات العقار من البطاقة"""
        try:
            data = {
                "العنوان": "غير معروف",
                "السعر (جنيه)": None,
                "سعر المتر (جنيه)": None,
                "المساحة (م²)": None,
                "الموقع": "غير معروف",
                "عدد الغرف": None,
                "عدد الحمامات": None,
                "عدد الصور": None,
                "رابط العقار": self.base_url,
                "وقت الاستخراج": self.run_timestamp
            }

            # استخراج العنوان - من الـ h2 أو من الـ title attribute
            try:
                title_elem = prop.find(
                    'h2', class_=re.compile(r'title|text-body'))
                if title_elem:
                    title = title_elem.get(
                        'title') or title_elem.get_text(strip=True)
                    if title and len(title) > 5:
                        data["العنوان"] = title
            except:
                pass

            # استخراج السعر الأساسي
            try:
                price_elem = prop.find(
                    'span', class_=re.compile(r'text-title|price'))
                if price_elem:
                    price_text = price_elem.get_text(strip=True)
                    data["السعر (جنيه)"] = self.clean_price(price_text)
            except:
                pass

            # استخراج سعر المتر
            try:
                price_per_meter_elem = prop.find(
                    'span', string=re.compile(r'ج/ م²'))
                if price_per_meter_elem:
                    price_meter_text = price_per_meter_elem.get_text(
                        strip=True)
                    data["سعر المتر (جنيه)"] = self.clean_price(
                        price_meter_text)
            except:
                pass

            # استخراج المساحة
            try:
                area_elem = prop.find('p', string=re.compile(r'متر²'))
                if area_elem:
                    area_text = area_elem.get_text(strip=True)
                    area_match = re.search(r'(\d+)\s*متر²', area_text)
                    if area_match:
                        data["المساحة (م²)"] = int(area_match.group(1))
            except:
                pass

            # استخراج الموقع
            try:
                location_elem = prop.find(
                    'p', class_=re.compile(r'text-caption'))
                if location_elem and location_elem.get('title'):
                    data["الموقع"] = location_elem.get('title')
                else:
                    # البحث عن نص يحتوي على موقع
                    location_texts = prop.find_all(
                        text=re.compile(r'القاهرة|التجمع|الحي|شارع|كمبوند'))
                    for text in location_texts:
                        if len(text.strip()) > 5:
                            data["الموقع"] = text.strip()
                            break
            except:
                pass

            # استخراج عدد الغرف والحمامات
            try:
                details_container = prop.find(
                    'div', class_=re.compile(r'flex items-center'))
                if details_container:
                    details = details_container.find_all(
                        'p', class_=re.compile(r'text-caption'))
                    for detail in details:
                        text = detail.get_text(strip=True)
                        if 'متر²' in text:
                            area_match = re.search(r'(\d+)', text)
                            if area_match and not data["المساحة (م²)"]:
                                data["المساحة (م²)"] = int(area_match.group(1))
                        elif text.isdigit():
                            # إذا كان رقم فقط، غالباً هو عدد الغرف أو الحمامات
                            num = int(text)
                            if not data["عدد الغرف"]:
                                data["عدد الغرف"] = num
                            elif not data["عدد الحمامات"]:
                                data["عدد الحمامات"] = num
            except:
                pass

            # استخراج عدد الصور
            try:
                images_elem = prop.find(
                    'span', class_=re.compile(r'text-white'))
                if images_elem and images_elem.get_text().isdigit():
                    data["عدد الصور"] = int(images_elem.get_text())
            except:
                pass

            # استخراج الرابط
            try:
                link_elem = prop.find('a', href=re.compile(r'/listing/'))
                if link_elem and link_elem.get('href'):
                    data["رابط العقار"] = "https://aqarmap.com.eg" + \
                        link_elem['href']
            except:
                pass

            return data if data["العنوان"] != "غير معروف" else None

        except Exception as e:
            logging.error(f"خطأ في استخراج بيانات العقار: {e}")
            return None

    def scrape_page(self, url):
        """استخراج البيانات من صفحة واحدة"""
        try:
            logging.info(f"📄 جاري تحميل الصفحة: {url}")
            self.driver.get(url)
            time.sleep(self.wait_time)

            # الحصول على محتوى الصفحة
            soup = BeautifulSoup(self.driver.page_source, "html.parser")

            # البحث عن بطاقات العقارات - التركيز على الـ listing-card
            property_cards = soup.find_all(
                'div', class_=re.compile(r'listing-card'))

            if not property_cards:
                logging.warning(
                    "⚠️ لم يتم العثور على أي بطاقات عقارات بـ class 'listing-card'")
                # محاولة البحث بأنماط أخرى
                property_cards = soup.find_all(
                    'div', class_=re.compile(r'card|property|listing'))
                logging.info(
                    f"🔍 تم العثور على {len(property_cards)} بطاقة بأنماط أخرى")

            logging.info(
                f"🏠 تم العثور على {len(property_cards)} بطاقة عقار في هذه الصفحة")

            # استخراج البيانات من كل بطاقة
            successful_extractions = 0
            for i, card in enumerate(property_cards):
                try:
                    property_data = self.extract_property_data(card)
                    if property_data:
                        self.results.append(property_data)
                        successful_extractions += 1
                        logging.info(
                            f"✅ {i+1}. {property_data['العنوان'][:50]}... - {property_data['السعر (جنيه)'] or '?'} جنيه")
                    else:
                        logging.warning(
                            f"❌ {i+1}. فشل في استخراج بيانات البطاقة")
                except Exception as e:
                    logging.error(f"❌ خطأ في معالجة البطاقة {i+1}: {e}")
                    continue

            logging.info(
                f"🎯 تم استخراج {successful_extractions} من أصل {len(property_cards)} عقار بنجاح")

            # البحث عن الصفحة التالية
            next_url = self.find_next_page(soup, url)
            return next_url

        except Exception as e:
            logging.error(f"🚨 خطأ في استخراج الصفحة: {e}")
            return None

    def find_next_page(self, soup, current_url):
        """البحث عن رابط الصفحة التالية"""
        try:
            # البحث في أزرار الترقيم
            pagination_links = soup.find_all('a', href=True)

            # استخراج رقم الصفحة الحالية
            current_page_match = re.search(r'page=(\d+)', current_url)
            current_page = int(current_page_match.group(1)
                               ) if current_page_match else 1

            # البحث عن رابط الصفحة التالية
            for link in pagination_links:
                href = link.get('href', '')
                text = link.get_text(strip=True)

                # البحث بالرقم التالي
                if f'page={current_page + 1}' in href:
                    next_url = "https://aqarmap.com.eg" + \
                        href if not href.startswith('http') else href
                    logging.info(f"🔗 تم العثور على الصفحة التالية: {next_url}")
                    return next_url

                # البحث بكلمة "التالي"
                if any(word in text.lower() for word in ['التالي', 'next', '›', '>']):
                    next_url = "https://aqarmap.com.eg" + \
                        href if not href.startswith('http') else href
                    logging.info(f"🔗 تم العثور على الصفحة التالية: {next_url}")
                    return next_url

            # إذا لم نجد، نجرب بناء الرابط يدوياً
            if current_page_match:
                next_page = current_page + 1
                next_url = re.sub(
                    r'page=\d+', f'page={next_page}', current_url)
                if next_url != current_url:
                    logging.info(f"🔗 بناء رابط الصفحة التالية: {next_url}")
                    return next_url
                else:
                    # إضافة parameter الصفحة إذا لم يكن موجود
                    separator = '&' if '?' in current_url else '?'
                    next_url = f"{current_url}{separator}page={next_page}"
                    logging.info(f"🔗 بناء رابط الصفحة التالية: {next_url}")
                    return next_url

            logging.info("⏹️ لم يتم العثور على صفحة تالية")
            return None

        except Exception as e:
            logging.error(f"خطأ في البحث عن الصفحة التالية: {e}")
            return None

    def scrape(self):
        """بدء عملية الاستخراج"""
        try:
            url = self.base_url
            for page_num in range(1, self.max_pages + 1):
                if not url:
                    logging.info("⏹️ لا توجد صفحات أخرى للمتابعة")
                    break

                logging.info(f"\n{'='*50}")
                logging.info(f"🎯 الصفحة {page_num}/{self.max_pages}")
                logging.info(f"{'='*50}")

                next_url = self.scrape_page(url)

                if next_url and next_url != url:
                    url = next_url
                else:
                    logging.info("⏹️ وصلنا لآخر صفحة")
                    break

                # انتظار بين الصفحات
                if page_num < self.max_pages:
                    logging.info(
                        f"⏳ انتظار {self.wait_time} ثواني قبل الصفحة التالية...")
                    time.sleep(self.wait_time)

            logging.info(
                f"\n✅ تم الانتهاء من الاستخراج. العدد الإجمالي: {len(self.results)} عقار")

        except Exception as e:
            logging.error(f"🚨 خطأ في عملية الاستخراج: {e}")
        finally:
            if hasattr(self, 'driver') and self.driver:
                self.driver.quit()
                logging.info("تم إغلاق المتصفح")

    def save_to_excel(self, filename="عقارات_القاهرة_الجديدة.xlsx"):
        """حفظ البيانات في ملف Excel"""
        if not self.results:
            logging.warning("⚠️ لا توجد بيانات لحفظها")
            return

        try:
            df = pd.DataFrame(self.results)

            # تنظيف البيانات وإزالة التكرارات
            initial_count = len(df)
            df = df.drop_duplicates(
                subset=['العنوان', 'السعر (جنيه)', 'الموقع'], keep='first')
            final_count = len(df)

            if initial_count != final_count:
                logging.info(f"🧹 تم إزالة {initial_count - final_count} تكرار")

            # إضافة عمود للملاحظات
            df['ملاحظات'] = ''

            # إعادة ترتيب الأعمدة
            column_order = [
                'العنوان', 'السعر (جنيه)', 'سعر المتر (جنيه)', 'المساحة (م²)', 'الموقع',
                'عدد الغرف', 'عدد الحمامات', 'عدد الصور',
                'رابط العقار', 'ملاحظات', 'وقت الاستخراج'
            ]

            # التأكد من وجود جميع الأعمدة
            for col in column_order:
                if col not in df.columns:
                    df[col] = None

            df = df[column_order]

            # حفظ في Excel
            df.to_excel(filename, index=False, engine='openpyxl')
            logging.info(f"💾 تم حفظ البيانات في: {filename}")

            # عرض إحصائيات
            self.show_statistics(df)

        except Exception as e:
            logging.error(f"🚨 خطأ في حفظ الملف: {e}")

    def show_statistics(self, df):
        """عرض إحصائيات عن البيانات"""
        print("\n" + "="*60)
        print("📊 إحصائيات البيانات المستخرجة")
        print("="*60)
        print(f"🏠 إجمالي العقارات: {len(df)}")

        if 'السعر (جنيه)' in df.columns:
            valid_prices = df['السعر (جنيه)'].dropna()
            if len(valid_prices) > 0:
                avg_price = valid_prices.mean()
                min_price = valid_prices.min()
                max_price = valid_prices.max()
                print(f"💰 متوسط السعر: {avg_price:,.0f} جنيه")
                print(f"📉 أقل سعر: {min_price:,.0f} جنيه")
                print(f"📈 أعلى سعر: {max_price:,.0f} جنيه")

        if 'المساحة (م²)' in df.columns:
            valid_areas = df['المساحة (م²)'].dropna()
            if len(valid_areas) > 0:
                avg_area = valid_areas.mean()
                print(f"📐 متوسط المساحة: {avg_area:.0f} م²")

        if 'سعر المتر (جنيه)' in df.columns:
            valid_meter_prices = df['سعر المتر (جنيه)'].dropna()
            if len(valid_meter_prices) > 0:
                avg_meter_price = valid_meter_prices.mean()
                print(f"📊 متوسط سعر المتر: {avg_meter_price:,.0f} جنيه")

        print("="*60)


# ------------------------------
# التشغيل الرئيسي
# ------------------------------
if __name__ == "__main__":
    # إضافة parameter page=1 للURL الأساسي
    base_url = "https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1&page=1"

    try:
        scraper = AqarMapScraper(base_url, max_pages=3, wait_time=4)
        scraper.scrape()
        scraper.save_to_excel()

        print("\n🎉 تم الانتهاء بنجاح!")
        print("📁 الملف المحفوظ: 'عقارات_القاهرة_الجديدة.xlsx'")

    except Exception as e:
        logging.error(f"❌ فشل التشغيل: {e}")

2025-09-29 23:51:37,995 - INFO - تم إعداد المتصفح بنجاح
2025-09-29 23:51:37,996 - INFO - 
2025-09-29 23:51:37,997 - INFO - 🎯 الصفحة 1/3
2025-09-29 23:51:37,997 - INFO - ==================================================
2025-09-29 23:51:37,997 - INFO - 📄 جاري تحميل الصفحة: https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1&page=1
2025-09-29 23:51:47,374 - INFO - 🏠 تم العثور على 22 بطاقة عقار في هذه الصفحة
2025-09-29 23:51:47,375 - INFO - ✅ 1. فيلا مستقلة ٢٦٠ متر ، كمبوند ذا وندر مارك ... - 31216800 جنيه
2025-09-29 23:51:47,376 - WARNING - ❌ 2. فشل في استخراج بيانات البطاقة
2025-09-29 23:51:47,377 - INFO - ✅ 3. شقة راقية ببيت الوطن  الحي الثامن ثاني صف من محور ... - 5382000 جنيه
2025-09-29 23:51:47,378 - INFO - ✅ 4. عيادة للبيع بجوار الجامعة الأمريكية ... - 11128000 جنيه
2025-09-29 23:51:47,378 - INFO - ✅ 5. شقه بحديقه خلفي +مخزن... - 4000000 جنيه
2025-09-29 23:51:47,379 - INFO - ✅ 6. ? *فرصة لا تُعوّض لامتلاك شقة مميزة في مدينتي – ال... - 12800000 جنيه
2025


📊 إحصائيات البيانات المستخرجة
🏠 إجمالي العقارات: 54
💰 متوسط السعر: 13,959,046 جنيه
📉 أقل سعر: 3,560,000 جنيه
📈 أعلى سعر: 32,634,381 جنيه

🎉 تم الانتهاء بنجاح!
📁 الملف المحفوظ: 'عقارات_القاهرة_الجديدة.xlsx'


In [ ]:
import time
import re
import logging
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os

# ------------------------------
# Logging
# ------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)


class AqarMapScraper:
    def __init__(self, base_url, max_pages=5, wait_time=5):
        self.base_url = base_url
        self.max_pages = max_pages
        self.wait_time = wait_time
        self.results = []
        self.run_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.driver = self.setup_driver()

    def setup_driver(self):
        """إعداد متصفح Chrome"""
        try:
            chrome_options = Options()
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument(
                "--disable-blink-features=AutomationControlled")
            chrome_options.add_experimental_option(
                "excludeSwitches", ["enable-automation"])
            chrome_options.add_experimental_option(
                'useAutomationExtension', False)
            chrome_options.add_argument("--lang=ar-EG")
            chrome_options.add_argument(
                "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

            driver = webdriver.Chrome(options=chrome_options)
            driver.set_page_load_timeout(30)
            driver.implicitly_wait(10)

            logging.info("تم إعداد المتصفح بنجاح")
            return driver

        except Exception as e:
            logging.error(f"فشل في إعداد المتصفح: {e}")
            raise

    def clean_price(self, price_text):
        """تنظيف نص السعر"""
        if not price_text:
            return None
        try:
            # إزالة كل ما ليس رقماً
            cleaned = re.sub(r'[^\d]', '', str(price_text))
            return int(cleaned) if cleaned else None
        except:
            return None

    def extract_all_properties_from_card(self, card):
        """استخراج جميع البيانات من بطاقة العقار"""
        try:
            data = {
                "العنوان": "غير معروف",
                "السعر (جنيه)": None,
                "سعر المتر (جنيه)": None,
                "المساحة (م²)": None,
                "الموقع": "غير معروف",
                "عدد الغرف": None,
                "عدد الحمامات": None,
                "عدد الصور": None,
                "رابط العقار": self.base_url,
                "وقت الاستخراج": self.run_timestamp
            }

            # استخراج العنوان
            try:
                title_elem = card.find(
                    'h2', class_=re.compile(r'title|text-body'))
                if title_elem:
                    title = title_elem.get(
                        'title') or title_elem.get_text(strip=True)
                    if title and len(title) > 5:
                        data["العنوان"] = title
            except:
                pass

            # استخراج السعر الأساسي
            try:
                price_spans = card.find_all(
                    'span', class_=re.compile(r'text-title'))
                for span in price_spans:
                    price_text = span.get_text(strip=True)
                    if 'جنيه' in price_text or any(char.isdigit() for char in price_text):
                        cleaned_price = self.clean_price(price_text)
                        if cleaned_price and cleaned_price > 1000:  # تأكد أنه سعر حقيقي
                            data["السعر (جنيه)"] = cleaned_price
                            break
            except:
                pass

            # استخراج سعر المتر
            try:
                # البحث عن النمط: "120,064 ج/ م²"
                price_per_meter_text = card.find(text=re.compile(r'ج/ م²'))
                if price_per_meter_text:
                    # البحث عن الرقم قبل "ج/ م²"
                    meter_match = re.search(
                        r'([\d,]+)\s*ج/ م²', price_per_meter_text)
                    if meter_match:
                        data["سعر المتر (جنيه)"] = self.clean_price(
                            meter_match.group(1))
            except:
                pass

            # استخراج المساحة
            try:
                # البحث في كل النصوص عن المساحة
                area_texts = card.find_all(text=re.compile(r'متر²|متر'))
                for text in area_texts:
                    area_match = re.search(r'(\d+)\s*متر²?', text)
                    if area_match:
                        data["المساحة (م²)"] = int(area_match.group(1))
                        break
            except:
                pass

            # استخراج الموقع
            try:
                location_elem = card.find('p', attrs={'title': True})
                if location_elem:
                    data["الموقع"] = location_elem.get('title')
                else:
                    # البحث عن نص يحتوي على موقع
                    location_texts = card.find_all(
                        text=re.compile(r'القاهرة|التجمع|الحي|شارع|كمبوند'))
                    for text in location_texts:
                        clean_text = text.strip()
                        if len(clean_text) > 5 and 'متر' not in clean_text:
                            data["الموقع"] = clean_text
                            break
            except:
                pass

            # استخراج عدد الغرف والحمامات
            try:
                # البحث عن الأيقونات والأرقام المرتبطة بها
                details_container = card.find(
                    'div', class_=re.compile(r'flex items-center'))
                if details_container:
                    # البحث عن جميع الأرقام في منطقة التفاصيل
                    numbers = re.findall(r'\d+', details_container.get_text())
                    icons = details_container.find_all(
                        'i', class_=re.compile(r'icon'))

                    # إذا وجدنا 3 أرقام، غالباً هم: المساحة، الغرف، الحمامات
                    if len(numbers) >= 3:
                        data["المساحة (م²)"] = data["المساحة (م²)"] or int(
                            numbers[0])
                        data["عدد الغرف"] = int(numbers[1])
                        data["عدد الحمامات"] = int(numbers[2])
            except:
                pass

            # استخراج عدد الصور
            try:
                images_span = card.find(
                    'span', class_=re.compile(r'text-white'))
                if images_span:
                    images_text = images_span.get_text(strip=True)
                    if images_text.isdigit():
                        data["عدد الصور"] = int(images_text)
            except:
                pass

            # استخراج الرابط
            try:
                link_elem = card.find('a', href=re.compile(r'/listing/'))
                if link_elem and link_elem.get('href'):
                    data["رابط العقار"] = "https://aqarmap.com.eg" + \
                        link_elem['href']
            except:
                pass

            # إذا لم نحصل على سعر، نحاول استخراجه من النص الكامل
            if not data["السعر (جنيه)"]:
                try:
                    full_text = card.get_text()
                    price_matches = re.findall(
                        r'(\d[\d,\.]+)\s*جني?ه?', full_text)
                    for match in price_matches:
                        cleaned = self.clean_price(match)
                        if cleaned and cleaned > 10000:  # سعر معقول للعقار
                            data["السعر (جنيه)"] = cleaned
                            break
                except:
                    pass

            # إذا لم نحصل على مساحة، نحاول من النص الكامل
            if not data["المساحة (م²)"]:
                try:
                    full_text = card.get_text()
                    area_match = re.search(r'(\d+)\s*متر²?', full_text)
                    if area_match:
                        data["المساحة (م²)"] = int(area_match.group(1))
                except:
                    pass

            return data if data["العنوان"] != "غير معروف" else None

        except Exception as e:
            logging.error(f"خطأ في استخراج بيانات العقار: {e}")
            return None

    def scrape_page(self, url):
        """استخراج البيانات من صفحة واحدة"""
        try:
            logging.info(f"📄 جاري تحميل الصفحة: {url}")
            self.driver.get(url)
            time.sleep(self.wait_time)

            # الحصول على محتوى الصفحة
            soup = BeautifulSoup(self.driver.page_source, "html.parser")

            # البحث عن بطاقات العقارات
            property_cards = soup.find_all(
                'div', class_=re.compile(r'listing-card'))

            if not property_cards:
                logging.warning("⚠️ لم يتم العثور على أي بطاقات عقارات")
                # حفظ HTML للتصحيح
                with open("debug_page.html", "w", encoding="utf-8") as f:
                    f.write(soup.prettify())
                logging.info("💾 تم حفظ صفحة HTML في debug_page.html للتحليل")
                return None

            logging.info(
                f"🏠 تم العثور على {len(property_cards)} بطاقة عقار في هذه الصفحة")

            # استخراج البيانات من كل بطاقة
            successful_extractions = 0
            for i, card in enumerate(property_cards):
                try:
                    property_data = self.extract_all_properties_from_card(card)
                    if property_data:
                        self.results.append(property_data)
                        successful_extractions += 1

                        # عرض ملخص عن العقار
                        price_str = f"{property_data['السعر (جنيه)']:,}" if property_data[
                            'السعر (جنيه)'] else "غير معروف"
                        area_str = f"{property_data['المساحة (م²)']} م²" if property_data[
                            'المساحة (م²)'] else "غير معروف"
                        logging.info(
                            f"✅ {i+1}. {property_data['العنوان'][:40]}... - {price_str} جنيه - {area_str}")
                    else:
                        logging.warning(
                            f"❌ {i+1}. فشل في استخراج بيانات البطاقة")
                except Exception as e:
                    logging.error(f"❌ خطأ في معالجة البطاقة {i+1}: {e}")
                    continue

            logging.info(
                f"🎯 تم استخراج {successful_extractions} من أصل {len(property_cards)} عقار بنجاح")

            # البحث عن الصفحة التالية
            next_url = self.find_next_page(soup, url)
            return next_url

        except Exception as e:
            logging.error(f"🚨 خطأ في استخراج الصفحة: {e}")
            return None

    def find_next_page(self, soup, current_url):
        """البحث عن رابط الصفحة التالية"""
        try:
            # البحث في أزرار الترقيم
            pagination_div = soup.find(
                'div', class_=re.compile(r'pagination|pages'))
            if pagination_div:
                links = pagination_div.find_all('a', href=True)
                for link in links:
                    href = link.get('href', '')
                    text = link.get_text(strip=True)

                    # البحث بالرقم التالي
                    current_page_match = re.search(r'page=(\d+)', current_url)
                    if current_page_match:
                        current_page = int(current_page_match.group(1))
                        if f'page={current_page + 1}' in href:
                            next_url = "https://aqarmap.com.eg" + \
                                href if not href.startswith('http') else href
                            logging.info(
                                f"🔗 تم العثور على الصفحة التالية: {next_url}")
                            return next_url

                    # البحث بكلمة "التالي"
                    if any(word in text.lower() for word in ['التالي', 'next', '›', '>']):
                        next_url = "https://aqarmap.com.eg" + \
                            href if not href.startswith('http') else href
                        logging.info(
                            f"🔗 تم العثور على الصفحة التالية: {next_url}")
                        return next_url

            # إذا لم نجد، نجرب بناء الرابط يدوياً
            current_page_match = re.search(r'page=(\d+)', current_url)
            if current_page_match:
                current_page = int(current_page_match.group(1))
                next_page = current_page + 1

                if 'page=' in current_url:
                    next_url = re.sub(
                        r'page=\d+', f'page={next_page}', current_url)
                else:
                    separator = '&' if '?' in current_url else '?'
                    next_url = f"{current_url}{separator}page={next_page}"

                logging.info(f"🔗 بناء رابط الصفحة التالية: {next_url}")
                return next_url

            logging.info("⏹️ لم يتم العثور على صفحة تالية")
            return None

        except Exception as e:
            logging.error(f"خطأ في البحث عن الصفحة التالية: {e}")
            return None

    def scrape(self):
        """بدء عملية الاستخراج"""
        try:
            url = self.base_url
            for page_num in range(1, self.max_pages + 1):
                if not url:
                    logging.info("⏹️ لا توجد صفحات أخرى للمتابعة")
                    break

                logging.info(f"\n{'='*60}")
                logging.info(f"🎯 الصفحة {page_num}/{self.max_pages}")
                logging.info(f"{'='*60}")

                next_url = self.scrape_page(url)

                if next_url and next_url != url:
                    url = next_url
                else:
                    logging.info("⏹️ وصلنا لآخر صفحة")
                    break

                # انتظار بين الصفحات
                if page_num < self.max_pages:
                    logging.info(
                        f"⏳ انتظار {self.wait_time} ثواني قبل الصفحة التالية...")
                    time.sleep(self.wait_time)

            logging.info(
                f"\n✅ تم الانتهاء من الاستخراج. العدد الإجمالي: {len(self.results)} عقار")

        except Exception as e:
            logging.error(f"🚨 خطأ في عملية الاستخراج: {e}")
        finally:
            if hasattr(self, 'driver') and self.driver:
                self.driver.quit()
                logging.info("تم إغلاق المتصفح")

    def save_to_excel(self, filename="عقارات_القاهرة_الجديدة.xlsx"):
        """حفظ البيانات في ملف Excel"""
        if not self.results:
            logging.warning("⚠️ لا توجد بيانات لحفظها")
            return

        try:
            df = pd.DataFrame(self.results)

            # تنظيف البيانات وإزالة التكرارات
            initial_count = len(df)
            df = df.drop_duplicates(
                subset=['العنوان', 'السعر (جنيه)', 'رابط العقار'], keep='first')
            final_count = len(df)

            if initial_count != final_count:
                logging.info(f"🧹 تم إزالة {initial_count - final_count} تكرار")

            # إضافة عمود للملاحظات
            df['ملاحظات'] = ''

            # إعادة ترتيب الأعمدة
            column_order = [
                'العنوان', 'السعر (جنيه)', 'سعر المتر (جنيه)', 'المساحة (م²)', 'الموقع',
                'عدد الغرف', 'عدد الحمامات', 'عدد الصور',
                'رابط العقار', 'ملاحظات', 'وقت الاستخراج'
            ]

            # التأكد من وجود جميع الأعمدة
            for col in column_order:
                if col not in df.columns:
                    df[col] = None

            df = df[column_order]

            # حفظ في Excel
            df.to_excel(filename, index=False, engine='openpyxl')
            logging.info(f"💾 تم حفظ البيانات في: {filename}")

            # عرض إحصائيات مفصلة
            self.show_detailed_statistics(df)

        except Exception as e:
            logging.error(f"🚨 خطأ في حفظ الملف: {e}")

    def show_detailed_statistics(self, df):
        """عرض إحصائيات مفصلة عن البيانات"""
        print("\n" + "="*70)
        print("📊 إحصائيات مفصلة عن البيانات المستخرجة")
        print("="*70)
        print(f"🏠 إجمالي العقارات: {len(df)}")

        # إحصائيات الأسعار
        if 'السعر (جنيه)' in df.columns:
            valid_prices = df['السعر (جنيه)'].dropna()
            if len(valid_prices) > 0:
                print(f"\n💰 إحصائيات الأسعار:")
                print(f"   • المتوسط: {valid_prices.mean():,.0f} جنيه")
                print(f"   • الأقل: {valid_prices.min():,.0f} جنيه")
                print(f"   • الأعلى: {valid_prices.max():,.0f} جنيه")
                print(f"   • العقارات ذات سعر معروف: {len(valid_prices)}")

        # إحصائيات المساحات
        if 'المساحة (م²)' in df.columns:
            valid_areas = df['المساحة (م²)'].dropna()
            if len(valid_areas) > 0:
                print(f"\n📐 إحصائيات المساحات:")
                print(f"   • المتوسط: {valid_areas.mean():.0f} م²")
                print(f"   • الأقل: {valid_areas.min():.0f} م²")
                print(f"   • الأعلى: {valid_areas.max():.0f} م²")
                print(f"   • العقارات ذات مساحة معروفة: {len(valid_areas)}")

        # إحصائيات سعر المتر
        if 'سعر المتر (جنيه)' in df.columns:
            valid_meter_prices = df['سعر المتر (جنيه)'].dropna()
            if len(valid_meter_prices) > 0:
                print(f"\n📊 إحصائيات سعر المتر:")
                print(
                    f"   • المتوسط: {valid_meter_prices.mean():,.0f} جنيه/م²")
                print(
                    f"   • العقارات بسعر متر معروف: {len(valid_meter_prices)}")

        # إحصائيات الغرف والحمامات
        rooms_count = df['عدد الغرف'].count()
        baths_count = df['عدد الحمامات'].count()
        images_count = df['عدد الصور'].count()

        print(f"\n🏘️ إحصائيات التفاصيل:")
        print(f"   • العقارات بعدد غرف معروف: {rooms_count}")
        print(f"   • العقارات بعدد حمامات معروف: {baths_count}")
        print(f"   • العقارات بعدد صور معروف: {images_count}")

        print("="*70)


# ------------------------------
# التشغيل الرئيسي
# ------------------------------
if __name__ == "__main__":
    base_url = "https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1&page=1"

    try:
        scraper = AqarMapScraper(base_url, max_pages=1, wait_time=4)
        scraper.scrape()
        scraper.save_to_excel()

        print("\n🎉 تم الانتهاء بنجاح!")
        print("📁 الملف المحفوظ: 'عقارات_القاهرة_الجديدة.xlsx'")

    except Exception as e:
        logging.error(f"❌ فشل التشغيل: {e}")

2025-09-30 00:00:28,790 - INFO - تم إعداد المتصفح بنجاح
2025-09-30 00:00:28,791 - INFO - 
2025-09-30 00:00:28,792 - INFO - 🎯 الصفحة 1/18
2025-09-30 00:00:28,792 - INFO - ============================================================
2025-09-30 00:00:28,792 - INFO - 📄 جاري تحميل الصفحة: https://aqarmap.com.eg/ar/for-sale/property-type/cairo/new-cairo/?byOwnerOnly=1&page=1
2025-09-30 00:00:35,232 - INFO - 🏠 تم العثور على 22 بطاقة عقار في هذه الصفحة
C:\Users\ASUS\AppData\Local\Temp\ipykernel_41448\852664863.py:116: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  price_per_meter_text = card.find(text=re.compile(r'ج/ م²'))
C:\Users\ASUS\AppData\Local\Temp\ipykernel_41448\852664863.py:130: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  area_texts = card.find_all(text=re.compile(r'متر²|متر'))
2025-09-30 00:00:35,234 - INFO - ✅ 1. فيلا مستقلة ٢٦٠ متر ، كمبوند ذا وندر مار... - 31,216,800 ج


📊 إحصائيات مفصلة عن البيانات المستخرجة
🏠 إجمالي العقارات: 345

💰 إحصائيات الأسعار:
   • المتوسط: 13,307,916 جنيه
   • الأقل: 680,000 جنيه
   • الأعلى: 157,685,000 جنيه
   • العقارات ذات سعر معروف: 345

📐 إحصائيات المساحات:
   • المتوسط: 4457 م²
   • الأقل: 23 م²
   • الأعلى: 501100 م²
   • العقارات ذات مساحة معروفة: 345

🏘️ إحصائيات التفاصيل:
   • العقارات بعدد غرف معروف: 0
   • العقارات بعدد حمامات معروف: 0
   • العقارات بعدد صور معروف: 342

🎉 تم الانتهاء بنجاح!
📁 الملف المحفوظ: 'عقارات_القاهرة_الجديدة.xlsx'


In [ ]:
import time
import re
import logging
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os

# ------------------------------
# Logging
# ------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)


class AqarMapDetailScraper:
    def __init__(self, wait_time=5):
        self.wait_time = wait_time
        self.results = []
        self.run_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.driver = self.setup_driver()
        self.total_properties_scraped = 0

    def setup_driver(self):
        """إعداد متصفح Chrome"""
        try:
            chrome_options = Options()
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument(
                "--disable-blink-features=AutomationControlled")
            chrome_options.add_experimental_option(
                "excludeSwitches", ["enable-automation"])
            chrome_options.add_experimental_option(
                'useAutomationExtension', False)
            chrome_options.add_argument("--lang=ar-EG")
            chrome_options.add_argument(
                "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

            driver = webdriver.Chrome(options=chrome_options)
            driver.set_page_load_timeout(30)
            driver.implicitly_wait(10)

            logging.info("تم إعداد المتصفح بنجاح")
            return driver

        except Exception as e:
            logging.error(f"فشل في إعداد المتصفح: {e}")
            raise

    def read_urls_from_file(self, filename="property_urls.txt"):
        """قراءة الروابط من ملف نصي"""
        try:
            with open(filename, 'r', encoding='utf-8') as file:
                urls = [line.strip() for line in file if line.strip()
                        and line.startswith('http')]

            logging.info(f"📖 تم قراءة {len(urls)} رابط من الملف: {filename}")
            return urls
        except FileNotFoundError:
            logging.error(f"⚠️ ملف {filename} غير موجود")
            return []
        except Exception as e:
            logging.error(f"🚨 خطأ في قراءة الملف: {e}")
            return []

    def clean_price(self, price_text):
        """تنظيف نص السعر"""
        if not price_text:
            return None
        try:
            # إزالة كل ما ليس رقماً
            cleaned = re.sub(r'[^\d]', '', str(price_text))
            return int(cleaned) if cleaned else None
        except:
            return None

    def extract_price_from_detail_page(self, soup):
        """استخراج السعر من صفحة التفاصيل"""
        try:
            # البحث عن السعر في عناصر مختلفة
            price_selectors = [
                'span[class*="price"]',
                'div[class*="price"]',
                'h2[class*="price"]',
                '.text-title',
                '[class*="amount"]'
            ]

            for selector in price_selectors:
                price_elements = soup.select(selector)
                for elem in price_elements:
                    price_text = elem.get_text(strip=True)
                    if any(char.isdigit() for char in price_text):
                        cleaned_price = self.clean_price(price_text)
                        if cleaned_price and cleaned_price > 1000:
                            return cleaned_price

            # البحث في النص الكامل للصفحة
            full_text = soup.get_text()
            price_matches = re.findall(r'(\d[\d,\.]+)\s*جني?ه?', full_text)
            for match in price_matches:
                cleaned = self.clean_price(match)
                if cleaned and cleaned > 10000:
                    return cleaned

            return None
        except Exception as e:
            logging.error(f"خطأ في استخراج السعر: {e}")
            return None

    def extract_area_from_detail_page(self, soup):
        """استخراج المساحة من صفحة التفاصيل"""
        try:
            # البحث عن عناصر المساحة باستخدام الأيقونات أو النص
            area_selectors = [
                'svg + p.text-body_1',  # أيقونة + نص المساحة
                '[class*="meter"]',
                '[class*="مساحة"]',
                '[class*="area"]'
            ]

            for selector in area_selectors:
                elements = soup.select(selector)
                for elem in elements:
                    text = elem.get_text(strip=True)
                    area_match = re.search(r'(\d+)\s*متر?²?', text)
                    if area_match:
                        return int(area_match.group(1))

            # البحث في النص الكامل
            full_text = soup.get_text()
            area_patterns = [
                r'(\d+)\s*متر?\s*²?',
                r'(\d+)\s*م²',
                r'المساحة[\s:]*(\d+)',
                r'مساحة[\s:]*(\d+)'
            ]

            for pattern in area_patterns:
                matches = re.findall(pattern, full_text, re.IGNORECASE)
                for match in matches:
                    if match.isdigit():
                        area = int(match)
                        if 20 <= area <= 10000:  # نطاق معقول للمساحة
                            return area

            return None
        except Exception as e:
            logging.error(f"خطأ في استخراج المساحة: {e}")
            return None

    def extract_rooms_baths_from_detail_page(self, soup):
        """استخراج عدد الغرف والحمامات من صفحة التفاصيل"""
        try:
            rooms = None
            baths = None

            # البحث عن الغرف باستخدام الأيقونات
            room_elements = soup.find_all(
                ['div', 'p'], string=re.compile(r'\d+\s*غرف'))
            for elem in room_elements:
                text = elem.get_text(strip=True)
                room_match = re.search(r'(\d+)\s*غرف', text)
                if room_match:
                    rooms = int(room_match.group(1))
                    break

            # إذا لم نجد، نبحث في النص الكامل
            if not rooms:
                full_text = soup.get_text()
                room_patterns = [
                    r'(\d+)\s*غرفة',
                    r'(\d+)\s*غرف',
                    r'غرف[\s:]*(\d+)',
                    r'عدد الغرف[\s:]*(\d+)'
                ]

                for pattern in room_patterns:
                    matches = re.findall(pattern, full_text, re.IGNORECASE)
                    for match in matches:
                        if match.isdigit():
                            rooms = int(match)
                            break
                    if rooms:
                        break

            # البحث عن الحمامات باستخدام الأيقونات
            bath_elements = soup.find_all(
                ['div', 'p'], string=re.compile(r'\d+\s*حمام'))
            for elem in bath_elements:
                text = elem.get_text(strip=True)
                bath_match = re.search(r'(\d+)\s*حمام', text)
                if bath_match:
                    baths = int(bath_match.group(1))
                    break

            # إذا لم نجد، نبحث في النص الكامل
            if not baths:
                full_text = soup.get_text()
                bath_patterns = [
                    r'(\d+)\s*حمام',
                    r'(\d+)\s*دورة مياه',
                    r'حمام[\s:]*(\d+)',
                    r'عدد الحمامات[\s:]*(\d+)'
                ]

                for pattern in bath_patterns:
                    matches = re.findall(pattern, full_text, re.IGNORECASE)
                    for match in matches:
                        if match.isdigit():
                            baths = int(match)
                            break
                    if baths:
                        break

            return rooms, baths
        except Exception as e:
            logging.error(f"خطأ في استخراج الغرف والحمامات: {e}")
            return None, None

    def extract_description(self, soup):
        """استخراج وصف الإعلان من الهيكل المحدد"""
        try:
            # الطريقة 1: البحث في الهيكل المحدد الذي قدمته
            description_section = soup.find(
                'section', class_=re.compile(r'gap-y-3x'))
            if description_section:
                # التحقق من وجود عنوان "وصف الإعلان"
                title_element = description_section.find(
                    'p', class_=re.compile(r'text-title_3'))
                if title_element and 'وصف الإعلان' in title_element.get_text():
                    # البحث عن محتوى الوصف في col-span-9
                    content_div = description_section.find(
                        'div', class_=re.compile(r'col-span-9'))
                    if content_div:
                        # استخراج النص من جميع عناصر span
                        span_elements = content_div.find_all('span')
                        if span_elements:
                            description_parts = []
                            for span in span_elements:
                                text = span.get_text(strip=True)
                                if text:
                                    description_parts.append(text)

                            if description_parts:
                                full_description = ' '.join(description_parts)
                                return full_description.strip()

            # الطريقة 2: البحث في عناصر الوصف الشائعة الأخرى
            description_selectors = [
                'div[class*="description"]',
                'p[class*="description"]',
                'div[class*="details"]',
                'div[class*="content"]',
                'section[class*="description"]',
                '.text-body',
                '[class*="وصف"]'
            ]

            for selector in description_selectors:
                elements = soup.select(selector)
                for elem in elements:
                    description_text = elem.get_text(strip=True)
                    if description_text and len(description_text) > 20:  # نص ذو معنى
                        return description_text[:1000]  # تحديد الطول

            # الطريقة 3: البحث في عناصر meta description
            meta_desc = soup.find('meta', attrs={'name': 'description'})
            if meta_desc and meta_desc.get('content'):
                desc_content = meta_desc['content']
                if len(desc_content) > 20:
                    return desc_content[:1000]

            return None
        except Exception as e:
            logging.error(f"خطأ في استخراج الوصف: {e}")
            return None

    def extract_property_type_and_date(self, soup):
        """استخراج نوع العقار والتاريخ من العنصر المحدد"""
        try:
            property_type = None
            date_text = None

            # البحث عن العنصر المحدد
            type_date_element = soup.find(
                'span', class_='text-caption text-gray__dark_1 flex flex-row')

            if type_date_element:
                # استخراج النص الكامل
                full_text = type_date_element.get_text(strip=True)

                # فصل النوع والتاريخ (يفترض أنهم مفصولين بنقطة)
                parts = full_text.split('·')
                if len(parts) >= 2:
                    property_type = parts[0].strip()
                    date_text = parts[1].strip()
                else:
                    # إذا لم يكن هناك فصل، نجرب استخراج النوع والتاريخ بطرق أخرى
                    property_type = full_text

                    # البحث عن نمط التاريخ (يوم شهر سنة)
                    date_pattern = r'(\d{1,2}\s+\S+\s+\d{4})'
                    date_match = re.search(date_pattern, full_text)
                    if date_match:
                        date_text = date_match.group(1)
                        # إزالة التاريخ من النص الأصلي لاستخراج النوع
                        property_type = re.sub(
                            date_pattern, '', full_text).strip()

            return property_type, date_text

        except Exception as e:
            logging.error(f"خطأ في استخراج نوع العقار والتاريخ: {e}")
            return None, None

    def extract_additional_details(self, soup):
        """استخراج التفاصيل الإضافية من كود HTML المقدم"""
        try:
            details = {
                "الدور": None,
                "نوع المعلن": None,
                "رقم الإعلان": None,
                "نوع العقار": None,
                "تاريخ النشر": None,
                "وصف الإعلان": None
            }

            # استخراج نوع العقار والتاريخ
            property_type, date_text = self.extract_property_type_and_date(
                soup)
            details["نوع العقار"] = property_type
            details["تاريخ النشر"] = date_text

            # استخراج وصف الإعلان
            description = self.extract_description(soup)
            details["وصف الإعلان"] = description

            # استخراج من grid التفاصيل
            detail_grid = soup.find('div', class_=re.compile(
                r'grid grid-cols-1 md:grid-cols-11'))
            if detail_grid:
                # البحث عن جميع العناصر في grid التفاصيل
                detail_items = detail_grid.find_all(
                    'div', class_=re.compile(r'flex px-1\.5x py-2x'))

                for item in detail_items:
                    # استخراج النوع والقيمة
                    label_elem = item.find(
                        'h4', class_=re.compile(r'text-body_2'))
                    value_elem = item.find(
                        'span', class_=re.compile(r'text-body_1'))

                    if label_elem and value_elem:
                        label = label_elem.get_text(strip=True)
                        value = value_elem.get_text(strip=True)

                        if label == "الدور":
                            details["الدور"] = value
                        elif label == "نوع المعلن":
                            details["نوع المعلن"] = value
                        elif label == "رقم الإعلان":
                            details["رقم الإعلان"] = value

            return details

        except Exception as e:
            logging.error(f"خطأ في استخراج التفاصيل الإضافية: {e}")
            return {
                "الدور": None,
                "نوع المعلن": None,
                "رقم الإعلان": None,
                "نوع العقار": None,
                "تاريخ النشر": None,
                "وصف الإعلان": None
            }

    def extract_location_from_detail_page(self, soup):
        """استخراج الموقع من صفحة التفاصيل"""
        try:
            # البحث في عناوين الموقع
            location_selectors = [
                '[class*="location"]',
                '[class*="address"]',
                '[class*="area"]',
                '.text-caption',
                'a[href*="/ar/for-rent/"] p'
            ]

            for selector in location_selectors:
                elements = soup.select(selector)
                for elem in elements:
                    location_text = elem.get_text(strip=True)
                    if location_text and len(location_text) > 5:
                        # تصفية النصوص القصيرة غير المفيدة
                        if any(word in location_text.lower() for word in ['القاهرة', 'الجديدة', 'التجمع', 'المعادي', 'الشيخ', 'مدينة', 'حي', 'شارع']):
                            return location_text

            # البحث في النص الكامل
            full_text = soup.get_text()
            location_keywords = ['القاهرة', 'الجديدة', 'التجمع', 'المعادي',
                                 'الشيخ زايد', 'مدينة نصر', 'المهندسين', 'مصر الجديدة']

            for keyword in location_keywords:
                if keyword in full_text:
                    # استخراج الجملة التي تحتوي على الكلمة المفتاحية
                    sentences = re.split(r'[.!?]', full_text)
                    for sentence in sentences:
                        if keyword in sentence and len(sentence) > 10 and len(sentence) < 200:
                            return sentence.strip()

            return "غير معروف"
        except Exception as e:
            logging.error(f"خطأ في استخراج الموقع: {e}")
            return "غير معروف"

    def extract_title_from_detail_page(self, soup, url):
        """استخراج العنوان من صفحة التفاصيل"""
        try:
            # البحث في عناوين h1
            title_elem = soup.find('h1')
            if title_elem:
                title = title_elem.get_text(strip=True)
                if title and len(title) > 5:
                    return title

            # البحث في عناوين h2
            title_elem = soup.find('h2')
            if title_elem:
                title = title_elem.get_text(strip=True)
                if title and len(title) > 5:
                    return title

            # استخراج من وصف meta
            meta_title = soup.find('meta', property='og:title')
            if meta_title and meta_title.get('content'):
                title = meta_title['content']
                if len(title) > 5:
                    return title

            # استخراج من الرابط إذا فشل كل شيء
            return f"عقار - {url.split('/')[-1]}"
        except Exception as e:
            logging.error(f"خطأ في استخراج العنوان: {e}")
            return f"عقار - {url.split('/')[-1]}"

    def scrape_property_detail(self, url):
        """استخراج البيانات من صفحة تفاصيل عقار واحدة"""
        try:
            logging.info(f"📄 جاري تحميل صفحة العقار: {url}")
            self.driver.get(url)
            time.sleep(self.wait_time)

            # الحصول على محتوى الصفحة
            soup = BeautifulSoup(self.driver.page_source, "html.parser")

            # استخراج البيانات الأساسية
            data = {
                "العنوان": self.extract_title_from_detail_page(soup, url),
                "السعر (جنيه)": self.extract_price_from_detail_page(soup),
                "المساحة (م²)": self.extract_area_from_detail_page(soup),
                "الموقع": self.extract_location_from_detail_page(soup),
                "رابط العقار": url,
                "وقت الاستخراج": self.run_timestamp
            }

            # استخراج الغرف والحمامات
            rooms, baths = self.extract_rooms_baths_from_detail_page(soup)
            data["عدد الغرف"] = rooms
            data["عدد الحمامات"] = baths

            # استخراج التفاصيل الإضافية
            additional_details = self.extract_additional_details(soup)
            data.update(additional_details)

            # التحقق من وجود بيانات أساسية
            if data["العنوان"] and (data["السعر (جنيه)"] or data["المساحة (م²)"]):
                self.results.append(data)
                self.total_properties_scraped += 1

                # عرض ملخص
                price_str = f"{data['السعر (جنيه)']:,}" if data['السعر (جنيه)'] else "غير معروف"
                area_str = f"{data['المساحة (م²)']} م²" if data['المساحة (م²)'] else "غير معروف"
                rooms_str = f"{data['عدد الغرف']} غرف" if data['عدد الغرف'] else "غير معروف"
                baths_str = f"{data['عدد الحمامات']} حمام" if data['عدد الحمامات'] else "غير معروف"

                logging.info(f"✅ تم استخراج العقار: {data['العنوان'][:50]}...")
                logging.info(
                    f"   💰 السعر: {price_str} جنيه | 📐 المساحة: {area_str}")
                logging.info(
                    f"   🛏️ الغرف: {rooms_str} | 🚿 الحمامات: {baths_str}")
                logging.info(f"   📍 الموقع: {data['الموقع'][:40]}...")

                # عرض التفاصيل الإضافية إذا وجدت
                additional_info = []
                if data['نوع العقار']:
                    additional_info.append(f"🏛️ النوع: {data['نوع العقار']}")
                if data['تاريخ النشر']:
                    additional_info.append(f"📅 التاريخ: {data['تاريخ النشر']}")
                if data['الدور']:
                    additional_info.append(f"🏢 الدور: {data['الدور']}")
                if data['وصف الإعلان']:
                    desc_preview = data['وصف الإعلان'][:80] + "..." if len(
                        data['وصف الإعلان']) > 80 else data['وصف الإعلان']
                    additional_info.append(f"📝 الوصف: {desc_preview}")

                if additional_info:
                    logging.info(
                        f"   📊 تفاصيل إضافية: {' | '.join(additional_info)}")

                return True
            else:
                logging.warning(f"⚠️ لم يتم استخراج بيانات كافية من: {url}")
                return False

        except Exception as e:
            logging.error(f"🚨 خطأ في استخراج صفحة العقار {url}: {e}")
            return False

    def scrape_urls(self, urls):
        """بدء عملية الاستخراج من قائمة الروابط"""
        try:
            total_urls = len(urls)
            successful_scrapes = 0

            logging.info(f"🎯 بدء استخراج {total_urls} عقار")

            for i, url in enumerate(urls, 1):
                logging.info(f"\n{'='*60}")
                logging.info(f"🏠 العقار {i} من {total_urls}")
                logging.info(f"{'='*60}")

                if self.scrape_property_detail(url.strip()):
                    successful_scrapes += 1

                # انتظار بين الطلبات
                if i < total_urls:
                    logging.info(
                        f"⏳ انتظار {self.wait_time} ثواني قبل العقار التالي...")
                    time.sleep(self.wait_time)

            logging.info(
                f"\n✅ تم الانتهاء من الاستخراج. النجاح: {successful_scrapes}/{total_urls}")

        except Exception as e:
            logging.error(f"🚨 خطأ في عملية الاستخراج: {e}")
        finally:
            if hasattr(self, 'driver') and self.driver:
                self.driver.quit()
                logging.info("تم إغلاق المتصفح")

    def create_statistics_dataframe(self, df):
        """إنشاء DataFrame للإحصائيات"""
        stats_data = []

        # إحصائيات عامة
        stats_data.append({"الإحصائية": "إجمالي العقارات المفحوصة",
                          "القيمة": len(df), "الوحدة": "عقار"})
        stats_data.append({"الإحصائية": "نسبة النجاح في الاستخراج",
                          "القيمة": f"{(len(df)/self.total_properties_scraped)*100:.1f}%", "الوحدة": ""})

        # إحصائيات الأسعار
        if 'السعر (جنيه)' in df.columns:
            valid_prices = df['السعر (جنيه)'].dropna()
            if len(valid_prices) > 0:
                stats_data.append(
                    {"الإحصائية": "متوسط السعر", "القيمة": f"{valid_prices.mean():,.0f}", "الوحدة": "جنيه"})
                stats_data.append(
                    {"الإحصائية": "أقل سعر", "القيمة": f"{valid_prices.min():,.0f}", "الوحدة": "جنيه"})
                stats_data.append(
                    {"الإحصائية": "أعلى سعر", "القيمة": f"{valid_prices.max():,.0f}", "الوحدة": "جنيه"})
                stats_data.append({"الإحصائية": "العقارات ذات سعر معروف", "القيمة": len(
                    valid_prices), "الوحدة": "عقار"})

        # إحصائيات المساحات
        if 'المساحة (م²)' in df.columns:
            valid_areas = df['المساحة (م²)'].dropna()
            if len(valid_areas) > 0:
                stats_data.append(
                    {"الإحصائية": "متوسط المساحة", "القيمة": f"{valid_areas.mean():.0f}", "الوحدة": "م²"})
                stats_data.append(
                    {"الإحصائية": "أقل مساحة", "القيمة": f"{valid_areas.min():.0f}", "الوحدة": "م²"})
                stats_data.append(
                    {"الإحصائية": "أكبر مساحة", "القيمة": f"{valid_areas.max():.0f}", "الوحدة": "م²"})
                stats_data.append({"الإحصائية": "العقارات ذات مساحة معروفة", "القيمة": len(
                    valid_areas), "الوحدة": "عقار"})

        # إحصائيات التفاصيل الإضافية
        if 'نوع العقار' in df.columns:
            valid_types = df['نوع العقار'].dropna()
            if len(valid_types) > 0:
                stats_data.append({"الإحصائية": "أنواع العقارات المختلفة", "القيمة": len(
                    valid_types.unique()), "الوحدة": "نوع"})

        if 'وصف الإعلان' in df.columns:
            valid_descriptions = df['وصف الإعلان'].dropna()
            stats_data.append({"الإحصائية": "العقارات بوصف معروف", "القيمة": len(
                valid_descriptions), "الوحدة": "عقار"})

        # إحصائيات الوقت
        stats_data.append({"الإحصائية": "وقت الاستخراج",
                          "القيمة": self.run_timestamp, "الوحدة": ""})

        return pd.DataFrame(stats_data)

    def save_to_excel(self, filename="عقارات_مفصلة_من_القاهرة_الجديدة.xlsx"):
        """حفظ البيانات في ملف Excel"""
        if not self.results:
            logging.warning("⚠️ لا توجد بيانات لحفظها")
            return

        try:
            df = pd.DataFrame(self.results)

            # تنظيف البيانات وإزالة التكرارات
            initial_count = len(df)
            df = df.drop_duplicates(subset=['رابط العقار'], keep='first')
            final_count = len(df)

            if initial_count != final_count:
                logging.info(f"🧹 تم إزالة {initial_count - final_count} تكرار")

            # إعادة ترتيب الأعمدة
            column_order = [
                'العنوان', 'السعر (جنيه)', 'المساحة (م²)', 'الموقع',
                'عدد الغرف', 'عدد الحمامات', 'نوع العقار', 'تاريخ النشر',
                'الدور', 'نوع المعلن', 'رقم الإعلان', 'وصف الإعلان',
                'رابط العقار', 'وقت الاستخراج'
            ]

            # التأكد من وجود جميع الأعمدة
            for col in column_order:
                if col not in df.columns:
                    df[col] = None

            df = df[column_order]

            # إنشاء DataFrame للإحصائيات
            stats_df = self.create_statistics_dataframe(df)

            # حفظ في Excel بشيتين منفصلين
            with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                # شيت البيانات الرئيسي
                df.to_excel(writer, sheet_name='العقارات', index=False)

                # شيت الإحصائيات
                stats_df.to_excel(writer, sheet_name='الإحصائيات', index=False)

                # تحسين عرض الأعمدة
                workbook = writer.book
                worksheet = writer.sheets['العقارات']

                column_widths = {
                    'A': 50,  # العنوان
                    'B': 20,  # السعر
                    'C': 15,  # المساحة
                    'D': 40,  # الموقع
                    'E': 15,  # الغرف
                    'F': 15,  # الحمامات
                    'G': 20,  # نوع العقار
                    'H': 20,  # تاريخ النشر
                    'I': 15,  # الدور
                    'J': 20,  # نوع المعلن
                    'K': 20,  # رقم الإعلان
                    'L': 60,  # وصف الإعلان
                    'M': 60,  # الرابط
                    'N': 20   # وقت الاستخراج
                }

                for col, width in column_widths.items():
                    worksheet.column_dimensions[col].width = width

                # تحسين عرض شيت الإحصائيات
                stats_worksheet = writer.sheets['الإحصائيات']
                stats_worksheet.column_dimensions['A'].width = 35
                stats_worksheet.column_dimensions['B'].width = 25
                stats_worksheet.column_dimensions['C'].width = 15

            logging.info(f"💾 تم حفظ البيانات في: {filename}")
            self.show_detailed_statistics(df)

        except Exception as e:
            logging.error(f"🚨 خطأ في حفظ الملف: {e}")

    def show_detailed_statistics(self, df):
        """عرض إحصائيات مفصلة عن البيانات"""
        print("\n" + "="*70)
        print("📊 إحصائيات مفصلة عن البيانات المستخرجة")
        print("="*70)
        print(f"🏠 إجمالي العقارات المستخرجة: {len(df)}")
        print(f"🔗 إجمالي الروابط المفحوصة: {self.total_properties_scraped}")

        # إحصائيات الأسعار
        if 'السعر (جنيه)' in df.columns:
            valid_prices = df['السعر (جنيه)'].dropna()
            if len(valid_prices) > 0:
                print(f"\n💰 إحصائيات الأسعار:")
                print(f"   • المتوسط: {valid_prices.mean():,.0f} جنيه")
                print(f"   • الأقل: {valid_prices.min():,.0f} جنيه")
                print(f"   • الأعلى: {valid_prices.max():,.0f} جنيه")
                print(f"   • العقارات ذات سعر معروف: {len(valid_prices)}")

        # إحصائيات المساحات
        if 'المساحة (م²)' in df.columns:
            valid_areas = df['المساحة (م²)'].dropna()
            if len(valid_areas) > 0:
                print(f"\n📐 إحصائيات المساحات:")
                print(f"   • المتوسط: {valid_areas.mean():.0f} م²")
                print(f"   • الأقل: {valid_areas.min():.0f} م²")
                print(f"   • الأعلى: {valid_areas.max():.0f} م²")
                print(f"   • العقارات ذات مساحة معروفة: {len(valid_areas)}")

        # إحصائيات التفاصيل الإضافية
        if 'نوع العقار' in df.columns:
            valid_types = df['نوع العقار'].dropna()
            if len(valid_types) > 0:
                print(f"\n🏛️ إحصائيات أنواع العقارات:")
                print(
                    f"   • أنواع العقارات: {', '.join(valid_types.unique()[:5])}")
                print(f"   • العقارات بنوع معروف: {len(valid_types)}")

        if 'وصف الإعلان' in df.columns:
            valid_descriptions = df['وصف الإعلان'].dropna()
            print(f"\n📝 إحصائيات الوصف:")
            print(f"   • العقارات بوصف معروف: {len(valid_descriptions)}")
            if len(valid_descriptions) > 0:
                avg_length = valid_descriptions.str.len().mean()
                print(f"   • متوسط طول الوصف: {avg_length:.0f} حرف")

        print("="*70)


# ------------------------------
# التشغيل الرئيسي
# ------------------------------
if __name__ == "__main__":
    scraper = AqarMapDetailScraper(wait_time=3)

    # قراءة الروابط من ملف نصي
    urls = scraper.read_urls_from_file("property_urls.txt")

    # إذا لم توجد روابط في الملف، استخدم القائمة الافتراضية
    if not urls:
        logging.info("📝 استخدام قائمة الروابط الافتراضية")
        urls = [
            "https://aqarmap.com.eg/ar/listing/5945347-for-rent-cairo-new-cairo-90th-street-northern-90th-street/",
            "https://aqarmap.com.eg/ar/listing/6378071-for-rent-cairo-new-cairo-compounds-district-5-the-plateau-district-5/",
            "https://aqarmap.com.eg/ar/listing/6362942-for-rent-cairo-new-cairo-madinaty-twelfth-zone-buildings-madinaty-b-12/",
            # ... أضف بقية الروابط هنا
        ]

    if urls:
        try:
            scraper.scrape_urls(urls)
            scraper.save_to_excel()

            print("\n🎉 تم الانتهاء بنجاح!")
            print("📁 الملف المحفوظ: 'عقارات_مفصلة_من_القاهرة_الجديدة.xlsx'")
            print("📊 يحتوي على شيتين: 'العقارات' و 'الإحصائيات'")
            print("🏢 البيانات المستخرجة تشمل:")
            print("   • العنوان والسعر والمساحة")
            print("   • عدد الغرف والحمامات")
            print("   • نوع العقار وتاريخ النشر")
            print("   • الدور")
            print("   • نوع المعلن ورقم الإعلان")
            print("   • وصف الإعلان (من الهيكل المحدد) 🆕")

        except Exception as e:
            logging.error(f"❌ فشل التشغيل: {e}")
    else:
        logging.error("❌ لا توجد روابط لاستخراج البيانات منها")